In [1]:
!pip install matplotlib torch_geometric torch gdown --quiet

In [ ]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [2]:
%cd hackaton/

/home/onyxia/work/DL-Hackathon/hackaton


In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [3]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 30 14:39 A
drwxr-sr-x 2 onyxia users 4.0K May 30 14:39 B
drwxr-sr-x 2 onyxia users 4.0K May 30 14:40 C
drwxr-sr-x 2 onyxia users 4.0K May 30 14:40 D


In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [5]:
# Modification og the class GraphDataset to get indexes

#import gzip
#import json
#import torch
#from torch_geometric.data import Dataset, Data
#from torch_geometric.loader import DataLoader

#class GraphDataset(Dataset):
#    def __init__(self, filename, transform=None, pre_transform=None):
#        self.raw = filename
#        self.num_graphs, self.graphs_dicts = self._count_graphs() 
#        super().__init__(None, transform, pre_transform)

#    def len(self):
#        return self.num_graphs  

#    # Addition of this function
#    def get(self, idx):
#        data = dictToGraphObject(self.graphs_dicts[idx])
#        data.idx = idx
#        return data

#    def _count_graphs(self):
#        with gzip.open(self.raw, "rt", encoding="utf-8") as f:
#            graphs_dicts = json.load(f)
#            return len(graphs_dicts), graphs_dicts

#def dictToGraphObject(graph_dict):
#    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
#    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict["edge_attr"] else None
#    num_nodes = graph_dict["num_nodes"]
#    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict["y"] is not None else None
#    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y)

In [6]:
# try to use this one 
import gzip
import json
import torch
from torch_geometric.data import Dataset, Data
import os
from tqdm import tqdm 
from torch_geometric.loader import DataLoader

class GraphDataset(Dataset):
    def __init__(self, filename, transform=None, pre_transform=None):
        self.raw = filename
        self.graphs = self.loadGraphs(self.raw)
        super().__init__(None, transform, pre_transform)

    def len(self):
        return len(self.graphs)

    def get(self, idx):
        return self.graphs[idx]

    @staticmethod
    def loadGraphs(path):
        print(f"Loading graphs from {path}...")
        print("This may take a few minutes, please wait...")
        with gzip.open(path, "rt", encoding="utf-8") as f:
            graphs_dicts = json.load(f)
        graphs = []
        for graph_dict in tqdm(graphs_dicts, desc="Processing graphs", unit="graph"):
            graphs.append(dictToGraphObject(graph_dict))
        return graphs



def dictToGraphObject(graph_dict):
    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict["edge_attr"] else None
    num_nodes = graph_dict["num_nodes"]
    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict["y"] is not None else None
    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y)

In [7]:

# Modification of the class GNN to get the embeddings, in order to build means of class embeddings

import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set
import torch.nn.functional as F
from torch_geometric.nn.inits import uniform

from src.conv import GNN_node, GNN_node_Virtualnode

class GNN(torch.nn.Module):

    def __init__(self, num_class, num_layer = 5, emb_dim = 300, 
                    gnn_type = 'gin', virtual_node = True, residual = False, drop_ratio = 0.5, JK = "last", graph_pooling = "mean"):
        '''
            num_tasks (int): number of labels to be predicted
            virtual_node (bool): whether to add virtual node or not
        '''

        super(GNN, self).__init__()

        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.emb_dim = emb_dim
        self.num_class = num_class
        self.graph_pooling = graph_pooling

        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        ### GNN to generate node embeddings
        if virtual_node:
            self.gnn_node = GNN_node_Virtualnode(num_layer, emb_dim, JK = JK, drop_ratio = drop_ratio, residual = residual, gnn_type = gnn_type)
        else:
            self.gnn_node = GNN_node(num_layer, emb_dim, JK = JK, drop_ratio = drop_ratio, residual = residual, gnn_type = gnn_type)


        ### Pooling function to generate whole-graph embeddings
        if self.graph_pooling == "sum":
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        elif self.graph_pooling == "attention":
            self.pool = GlobalAttention(gate_nn = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, 1)))
        elif self.graph_pooling == "set2set":
            self.pool = Set2Set(emb_dim, processing_steps = 2)
        else:
            raise ValueError("Invalid graph pooling type.")

        if graph_pooling == "set2set":
            self.graph_pred_linear = torch.nn.Linear(2*self.emb_dim, self.num_class)
        else:
            self.graph_pred_linear = torch.nn.Linear(self.emb_dim, self.num_class)

    def forward(self, batched_data, return_embeddings=False):
        h_node = self.gnn_node(batched_data)
        h_graph = self.pool(h_node, batched_data.batch)  # les embeddings de graphes
        logits = self.graph_pred_linear(h_graph)

        if return_embeddings:
            return logits, F.normalize(h_graph, p=2, dim=1)  # on retourne aussi les embeddings normalisés si demandé
        else:
            return logits


In [8]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [9]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [10]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [11]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [12]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [13]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (GCOD)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [14]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/A/train.json.gz
Path to the test dataset [None]:  datasets/A/test.json.gz
Number of checkpoints to save during training [None]:  20
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn
Dropout ratio [0.0]:  0.5
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  600
Baseline mode: 1 (CE), 2 (GCOD) [1]:  2
Noise probability p (used if baseline_mode=2) [0.2]:  


Arguments received:
train_path: datasets/A/train.json.gz
test_path: datasets/A/test.json.gz
num_checkpoints: 20
device: 1
gnn: gcn
drop_ratio: 0.5
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 600
baseline_mode: 2
noise_prob: 0.2


In [15]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    # We initializa criterion and optimizers in the cell after the definition of full_dataset, as we need train_size
    pass
else:
    criterion = torch.nn.CrossEntropyLoss()

In [16]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [17]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [18]:
# We define a class to give indexes for the train set (subset of the full set (train and val))
class IndexedSubDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, i):
        data = self.subset[i]
        data.idx = i
        return data

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCODLoss(nn.Module):
    #def __init__(self, num_samples, num_classes=6, alpha=1.0, beta=1.0):
    def __init__(self, num_classes=6, alpha=1.0, beta=1.0):
        super().__init__()
        self.num_classes = num_classes
        self.alpha = alpha
        self.beta = beta


    def forward(self, logits, u_batch, y_onehot, y_soft, a_train):
        """
        logits: (B, C)
        u_batch: (B,)
        y_onehot: (B, C)
        y_soft: (B, C)
        a_train: float
        """
        B, C = logits.shape

        u_diag_y = u_batch.unsqueeze(1) * y_onehot  # (B, C)

        # L1: weighted cross-entropy with soft labels
        logits_mod = logits + a_train * u_diag_y
        L1 = F.cross_entropy(logits_mod, y_soft.argmax(dim=1))

        # L2: squared norm alignment
        y_pred_onehot = torch.zeros_like(y_onehot)
        y_pred_onehot.scatter_(1, logits.argmax(dim=1, keepdim=True), 1)
        L2 = (1 / C) * ((y_pred_onehot + u_diag_y - y_onehot).pow(2).sum(dim=1).mean())

        # L3: KL regularization
        # Get fθ(ZB) @ yB^T → logits of the true class for each sample
        # logits: (B, C), y_onehot: (B, C)
        eps = 1e-12
        # Step 1: Matrix multiplication (B x C) @ (C x B) = (B x B)
        logits_yT = torch.matmul(logits, y_onehot.T)  # (B, B)
    
        # Step 2: Extract diagonal
        diag_logits_y = torch.diag(logits_yT)  # (B,)
    
        # Step 3: Softmax + log → log P
        log_P = F.log_softmax(diag_logits_y, dim=0)
        P = torch.exp(log_P)
    
        # Step 4: Softmax of -log(u)
        log_u = torch.log(u_batch.clamp(min=eps))
        Q = F.softmax(-log_u, dim=0)
        log_Q = torch.log(Q + eps)
    
        # Step 5: KL(P || Q)
        L3 = (1 - a_train) * torch.sum(P * (log_P - log_Q))


        return L1 + L3, L2

In [20]:
def train(train_loader, model, optimizer_theta, optimizer_u, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        batch = batch.to(device)

        optimizer_theta.zero_grad()
        optimizer_u.zero_grad()

        logits, embeddings = model(batch, return_embeddings=True) 
        y = batch.y
        y_onehot = F.one_hot(y, num_classes=criterion.num_classes).float()

        # Compute y_soft 
        # See paper : Wani, F. A., Bucarelli, M. S., and Silvestri, F. (2023).
        # Combining distance to class centroids and outlier discounting for improved learning with noisy labels 
        
        # Compute embeddings centroïds for each class 
        class_means = {}
        for c in torch.unique(y):
            mask = (y == c)
            class_embs = embeddings[mask]
            if class_embs.size(0) > 0:
                mean_emb = class_embs.mean(dim=0)
                norm_mean_emb = mean_emb / (mean_emb.norm(p=2) + 1e-12)  # normalize
                class_means[c.item()] = norm_mean_emb  # shape: [embedding_dim]
        
        # y_soft initialized to zero
        y_soft = torch.zeros_like(y_onehot)  # shape: [batch_size, num_classes]
        
        for i in range(len(y)):
            emb_i = embeddings[i]
            norm_emb_i = emb_i / (emb_i.norm(p=2) + 1e-12)
            class_idx = y[i].item()
        
            if class_idx in class_means:
                sim = torch.dot(norm_emb_i, class_means[class_idx])
                sim = torch.clamp(sim, min=0.0)  # ReLU
                y_soft[i, class_idx] = sim
            else:
                y_soft[i, class_idx] = 0.0



        # Get indices
        graph_indices = batch.idx.to(device)
        u_batch = u[graph_indices]  # 🔁 u est passé de l'extérieur

        #  Clamp u between 0 and 1 (u is a confidence index for the level of noise of the label)
        u.data.clamp_(0, 1)

        # predictions for accuracy
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
        train_acc = correct / total # train_acc = a_train in the paper 'Robustness of Graph Classification ...'

        # losses
        loss_theta, loss_u = criterion(logits, u_batch, y_onehot, y_soft, train_acc)

        # Backpropagation
        loss_theta.backward(retain_graph=True)
        optimizer_theta.step()

        loss_u.backward()
        optimizer_u.step()

        #  Clamp u between 0 and 1 (u is a confidence index for the level of noise of the label)
        u.data.clamp_(0, 1)


        total_loss += loss_theta.item()

    avg_loss = total_loss / len(train_loader)
    final_train_acc = correct / total
    return avg_loss, final_train_acc


In [21]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_dataset = IndexedSubDataset(train_dataset)

    if args.baseline_mode == 2:

        u = nn.Parameter(torch.zeros(len(train_dataset), device=device))
        optimizer_u = torch.optim.Adam([u], lr=1)
        optimizer_theta = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

        #scheduler = ReduceLROnPlateau(
        #optimizer_theta, 
        #mode='max',           # maximiser val_acc
        #factor=0.5,           # réduction du LR par 2
        #patience=5,           # n epochs sans amélioration
        #threshold=1e-3,       # tolérance d’amélioration
        #min_lr=1e-6           # LR minimum
        #)


        
        criterion = GCODLoss(num_classes=6, alpha=1.0, beta=1.0)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)  # shuffle=False mandatory, not to loose indexes ?
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer_theta, optimizer_u, criterion, device
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        # 🔽 Appel du scheduler avec la métrique à surveiller
        #scheduler.step(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Loading graphs from datasets/A/train.json.gz...
This may take a few minutes, please wait...


Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 14.40batch/s]
Epoch 1/600, Loss: 1.6689, Train Acc: 0.3546, Val Acc: 0.4145


Epoch 1/600, Loss: 1.6689, Train Acc: 0.3546, Val Acc: 0.4145
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_A_best.pth


Iterating eval graphs: 100%|██████████| 71/71 [00:04<00:00, 14.42batch/s]
Epoch 2/600, Loss: 5.0540, Train Acc: 0.3733, Val Acc: 0.4317


Epoch 2/600, Loss: 5.0540, Train Acc: 0.3733, Val Acc: 0.4317
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_A_best.pth


KeyboardInterrupt: 

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

# B

In [ ]:
args.train_path = 'datasets/B/train.json.gz'
args.test_path = 'datasets/B/test.json.gz'

In [ ]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    # We initializa criterion and optimizers in the cell after the definition of full_dataset, as we need train_size
    pass
else:
    criterion = torch.nn.CrossEntropyLoss()

In [ ]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [ ]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_dataset = IndexedSubDataset(train_dataset)

    if args.baseline_mode == 2:

        u = nn.Parameter(torch.zeros(len(train_dataset), device=device))
        optimizer_u = torch.optim.Adam([u], lr=1)
        optimizer_theta = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

        #scheduler = ReduceLROnPlateau(
        #optimizer_theta, 
        #mode='max',           # maximiser val_acc
        #factor=0.5,           # réduction du LR par 2
        #patience=5,           # n epochs sans amélioration
        #threshold=1e-3,       # tolérance d’amélioration
        #min_lr=1e-6           # LR minimum
        #)


        
        criterion = GCODLoss(num_classes=6, alpha=1.0, beta=1.0)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)  # shuffle=False mandatory, not to loose indexes ?
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer_theta, optimizer_u, criterion, device
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        # 🔽 Appel du scheduler avec la métrique à surveiller
        #scheduler.step(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

# C

In [ ]:
args.train_path = 'datasets/C/train.json.gz'
args.test_path = 'datasets/C/test.json.gz'

In [ ]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    # We initializa criterion and optimizers in the cell after the definition of full_dataset, as we need train_size
    pass
else:
    criterion = torch.nn.CrossEntropyLoss()

In [ ]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [ ]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_dataset = IndexedSubDataset(train_dataset)

    if args.baseline_mode == 2:

        u = nn.Parameter(torch.zeros(len(train_dataset), device=device))
        optimizer_u = torch.optim.Adam([u], lr=1)
        optimizer_theta = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

        #scheduler = ReduceLROnPlateau(
        #optimizer_theta, 
        #mode='max',           # maximiser val_acc
        #factor=0.5,           # réduction du LR par 2
        #patience=5,           # n epochs sans amélioration
        #threshold=1e-3,       # tolérance d’amélioration
        #min_lr=1e-6           # LR minimum
        #)


        
        criterion = GCODLoss(num_classes=6, alpha=1.0, beta=1.0)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)  # shuffle=False mandatory, not to loose indexes ?
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer_theta, optimizer_u, criterion, device
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        # 🔽 Appel du scheduler avec la métrique à surveiller
        #scheduler.step(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

# D

In [ ]:
args.train_path = 'datasets/D/train.json.gz'
args.test_path = 'datasets/D/test.json.gz'

In [ ]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    # We initializa criterion and optimizers in the cell after the definition of full_dataset, as we need train_size
    pass
else:
    criterion = torch.nn.CrossEntropyLoss()

In [ ]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [ ]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_dataset = IndexedSubDataset(train_dataset)

    if args.baseline_mode == 2:

        u = nn.Parameter(torch.zeros(len(train_dataset), device=device))
        optimizer_u = torch.optim.Adam([u], lr=1)
        optimizer_theta = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

        #scheduler = ReduceLROnPlateau(
        #optimizer_theta, 
        #mode='max',           # maximiser val_acc
        #factor=0.5,           # réduction du LR par 2
        #patience=5,           # n epochs sans amélioration
        #threshold=1e-3,       # tolérance d’amélioration
        #min_lr=1e-6           # LR minimum
        #)


        
        criterion = GCODLoss(num_classes=6, alpha=1.0, beta=1.0)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)  # shuffle=False mandatory, not to loose indexes ?
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer_theta, optimizer_u, criterion, device
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        # 🔽 Appel du scheduler avec la métrique à surveiller
        #scheduler.step(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

# Export

In [ ]:
import tarfile
import os

def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.

    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")

# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/submission"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/submission.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

In [ ]:
# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/logs"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/logs.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

In [ ]:
# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/checkpoints"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/checkpoints.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)